In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [2]:
(trainX, trainY), (testX, testY) = cifar10.load_data()


In [3]:
# Convertendo valores dos pixels em float
trainX = trainX.astype('float32')
testX = testX.astype('float32')

# normalização para escala [0-1]
trainX = trainX / 255.0
testX = testX / 255.0

# transformando a variável alvo (target) para uma codificação one hot
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [10]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
## Configurar o encerramento do treinamento quando o val_loss nao melhora
from tensorflow.python.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [12]:
scores = []
histories = []

# definindo a validação k-fold
kfold = KFold(5, shuffle=True, random_state=1)

# loop para as k-folds (cada fold usa subconjuntos de treino e teste diferentes)
for train_ix, test_ix in kfold.split(trainX):

    model = define_model()

    # recorta dados de acordo com índices da k-fold
    train_data, train_target, val_data, val_target = trainX[train_ix], trainY[train_ix], trainX[test_ix], trainY[test_ix]

    # treinamento do modelo
    history = model.fit(train_data, train_target,
                        epochs=10, batch_size=32,
                        validation_data=(val_data, val_target),
                        verbose=1)

    # desempenho do modelo
    _, acc = model.evaluate(val_data, val_target, verbose=0)
    print('> %.3f' % (acc * 100.0))

    # armazena resultados de cada modelo treinado dentro da k-fold
    scores.append(acc)
    histories.append(history)

print('Acurácia: média=%.3f desvio=%.3f' % (np.mean(scores)*100, np.std(scores)*100))
plt.boxplot(scores)
plt.show()

Epoch 1/10
1250/1250 [==============================] - 90s 72ms/step - loss: 1.3972 - accuracy: 0.5060 - val_loss: 1.2322 - val_accuracy: 0.5757
Epoch 2/10
1250/1250 [==============================] - 100s 80ms/step - loss: 1.0268 - accuracy: 0.6409 - val_loss: 1.5689 - val_accuracy: 0.4965
Epoch 3/10
1250/1250 [==============================] - 95s 76ms/step - loss: 0.8316 - accuracy: 0.7070 - val_loss: 1.3791 - val_accuracy: 0.5691
Epoch 4/10
1250/1250 [==============================] - 96s 77ms/step - loss: 0.6605 - accuracy: 0.7698 - val_loss: 1.2799 - val_accuracy: 0.5858
Epoch 5/10
1250/1250 [==============================] - 99s 79ms/step - loss: 0.5106 - accuracy: 0.8217 - val_loss: 1.4809 - val_accuracy: 0.5988
Epoch 6/10
1250/1250 [==============================] - 90s 72ms/step - loss: 0.3873 - accuracy: 0.8663 - val_loss: 1.6515 - val_accuracy: 0.5910
Epoch 7/10
1250/1250 [==============================] - 87s 70ms/step - loss: 0.2981 - accuracy: 0.8976 - val_loss: 2.3197 

KeyboardInterrupt: 